In [1]:
import pandas as pd
import numpy as np

In [2]:
file_url = 'https://raw.githubusercontent.com/AGilarde12/Mall-Customer-Segmentation/master/Data/Mall_Customers.csv'

In [3]:
df = pd.read_csv(file_url)

In [4]:
#df.to_csv('../data/raw/Mall_Customers.csv',index=False)

In [5]:
#df = pd.read_csv('../data/raw/Mall_Customers.csv')

In [6]:
df_cleaned = df.copy()

In [7]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [8]:
ohe = OneHotEncoder(sparse=False, drop='first')

In [9]:
df_cleaned['Gender'] = ohe.fit_transform(df_cleaned[['Gender']])

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
target = df.pop('Spending Score (1-100)')

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df_cleaned, target, test_size=0.2, shuffle = True, random_state = 8)

In [13]:
import mlflow
import mlflow.sklearn

In [14]:
mlflow.set_tracking_uri('http://mlflow:5000')

In [15]:
mlflow.set_experiment('xgboost_spending')

INFO: 'xgboost_spending' does not exist. Creating a new experiment


In [16]:
run = mlflow.start_run()

In [17]:
mlflow.set_tag("model.description", "RandomForest with default hyperparameter")

In [18]:
mlflow.set_tag("model.version", "0.1")

In [19]:
mlflow.sklearn.autolog()

In [20]:
from sklearn.ensemble import RandomForestRegressor

rf1 = RandomForestRegressor(random_state=8)

In [21]:
rf1.fit(X_train, y_train)

2021/03/07 06:10:10 WARNING mlflow.utils.autologging_utils: MLflow issued a warning during sklearn autologging: "/opt/conda/lib/python3.7/site-packages/mlflow/models/signature.py:123: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=8, verbose=0, warm_start=False)

In [22]:
from mlflow.models.signature import infer_signature

In [23]:
signature = infer_signature(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/mlflow/models/signature.py:124: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  outputs = _infer_schema(model_output) if model_output is not None else None


In [24]:
mlflow.sklearn.log_model(rf1, artifact_path="model", signature=signature, registered_model_name="sklearn-rf-spending") 

Successfully registered model 'sklearn-rf-spending'.
2021/03/07 06:10:48 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sklearn-rf-spending, version 1
Created version '1' of model 'sklearn-rf-spending'.


In [25]:
mlflow.end_run()